# Objetivo do código
Enriquecer uma base de até 700 CNPJs com pontos, barras e traços, enviados por meio de um arquivo CSV para adicionar informações de data de abertura, natureza jurídica, capital social, faturamento estimado, porte, código da atividade primária, código da atividade secundária 1, código da atividade secundária 2, código da atividade secundária 3, ..., código da atividade secundária 99.

# Como o código funciona
Usaremos a base de informações de CNPJ da API da receita federal pelo link https://www.receitaws.com.br/v1/cnpj/{cnpj}. A API é gratúita mas tem limitação de 720 solicitações por dia e até 3 consultas por segundo por IP. Portanto, usaremos a biblioteca Time para não realizar todas as solicitações de uma só vez


# Resultado do código
Uma planilha CSV com a informação de CNPJ, data de abertura, natureza jurídica, capital social, faturamento estimado, porte, código da atividade primária, código da atividade secundária 1, código da atividade secundária 2, código da atividade secundária 3, ..., código da atividade secundária 99.

**Código abaixo:**
Importe as bibliotecas necessárias: pandas para ler o arquivo CSV com os CNPJs, requests para fazer as chamadas à API, time para esperar um intervalo entre as chamadas

In [41]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import os

**Código abaixo:**
Defina a URL base da API do ReceitaWS e o token de acesso.

In [42]:
url_base = 'https://www.receitaws.com.br/v1/cnpj/'
token = 'SEU TOKEN AQUI'
headers = {'Authorization': 'Bearer ' + token}

**Código abaixo:**
Fazer um teste se a requisição está funcionando

In [43]:
cnpj_1 = '36073794000160'
test_response = requests.get(url_base + cnpj_1, headers)
test_response

<Response [200]>

**Código abaixo:**
Cria um caminho absoluto para o arquivo de entrada e um caminho absoluto para o arquivo de saída

In [44]:
# Obter o diretório atual de trabalho do notebook voltando 1x para o diretório pai
dir_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# construir o caminho absoluto para o arquivo de entrada
input_path = os.path.join(dir_path, 'arquivos_entrada', 'cnpjs_bruto.csv')

# construir o caminho absoluto para o arquivo de saída
output_path = os.path.join(dir_path, 'arquivos_saida', 'cnpjs_enriquecidos.csv')

**Código abaixo:**
Ler arquivo CSV e armazenar os CNPJs em uma lista

In [45]:
path = '/Users/nicolasstelatto/Energizou Dropbox/Nicolas Stelatto/Mac/Documents/Python Notebooks/Enrich CNPJs/Dados Brutos/CNPJs para testar.csv'
cnpjs_df = pd.read_csv(input_path, delimiter = ";")
cnpjs_list = list(cnpjs_df['CNPJs'])

**Código abaixo:**
Criar função **clean_cnpj** para limpar o cnpj com barras e pontos e retornar uma string somente com os números

In [46]:
def clean_cnpj(cnpj):
    step1 = cnpj.replace('.', '').replace('/', '').replace('-', '')
    if len(step1) < 14:
        step2 = step1.zfill(14)
        return step2
    else:
        return step1

**Código abaixo:**
Cria uma lista com os CNPJs limpos chamanda **cnpjs_list_clean**

In [47]:
cnpjs_list_clean = []
for element in cnpjs_list:
    cnpjs_list_clean.append(clean_cnpj(element))
cnpjs_list_clean

['32190231000100',
 '07852102000184',
 '36619769000130',
 '36583766000193',
 '08773135000100',
 '63310411001418',
 '19675900000167',
 '17838938000188',
 '45985371000108',
 '08014346000150']

**Código abaixo:**
Cria a função **consultar_cnpj** que faz a solicitação para a API por meio de um CNPJ especificado e retorna uma lista com todas as informações relevantes daquele CNPJ.

Essa função segue também a estrutura de que ela só irá ser executada se o tempo da última request e o tempo de request atual for maior ou igual a 20 segundos. Se for menor, usamos uma função que espera até que chegue em 20 segundos. Usamos essa regra para que não tenha problemas com a API, dado que o limite são 3 requisições por minuto.

In [54]:
last_request_time = None # variável global para armazenar o último momento em que uma requisição foi enviada
def consultar_cnpj(cnpj):
    global last_request_time    # indicando que estamos usando a variável global
    
    # Define o intervalo de tempo mínimo entre requisições (em segundos)
    request_interval = 30
    
    # Se é a primeira requisição, seta o last_request_time para o momento atual
    if not last_request_time:
        last_request_time = time.time()
    else:
        # Calcula o tempo decorrido desde a última requisição
        elapsed_time = time.time() - last_request_time
        
        # Verifica se o tempo decorrido é menor que o intervalo mínimo entre requisições
        if elapsed_time < request_interval:
            # Se for menor, pausa a execução do código pelo tempo restante necessário
            time.sleep(request_interval - elapsed_time)
        
        # Atualiza o last_request_time com o momento atual
        last_request_time = time.time()
    
    
    
    #código da requisição aqui
    url = url_base + cnpj
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        
        cnpj = data['cnpj']
        abertura = data['abertura']
        situacao = data['situacao']
        tipo = data['tipo']
        nome = data['nome']
        porte = data['porte']
        uf = data['uf']
        capital_social = data['capital_social']
        status = data['status']
        natureza_juridica = data['natureza_juridica']
        atividade_principal = data['atividade_principal'][0]['code']
        
        atividade_secundaria = []
        for atividade in data['atividades_secundarias']:
            atividade_secundaria.append(atividade['code'])
        
        while len(atividade_secundaria) < 99:
            atividade_secundaria.append(None)
        
        return cnpj, abertura, situacao, tipo, nome, porte, uf, capital_social, status, natureza_juridica, atividade_principal, *atividade_secundaria
 
    else:
        return None

**Código abaixo:**
Cria uma lista de listas vazia para armazenar as informações de todas as empresas.
Cria um loop que itera sobre a lista de CNPJs para chamar a função criada anteriormente e salvar a lista de informações na variável empresas. Ele imprime o valor de CNPJ de cada requisição para garantir que o Loop está funcionando

In [53]:
empresas = []
for cnpj in cnpjs_list_clean:
    # Faz a consulta à API
    info = consultar_cnpj(cnpj)
    if info:
        empresas.append(info)
        print(info[0])

32.190.231/0001-00
07.852.102/0001-84
36.619.769/0001-30
36.583.766/0001-93
08.773.135/0001-00
63.310.411/0014-18
19.675.900/0001-67
17.838.938/0001-88
45.985.371/0001-08
08.014.346/0001-50


**Código abaixo:**
Salva as informações em um DataFrame e em um novo arquivo CSV no Desktop do Mac

In [ ]:
empresas_df = pd.DataFrame(empresas, columns=['cnpj', 'data_abertura', 'situacao_fiscal', 'tipo', 'razao_social', 'porte', 'uf', 'capital_social', 'status', 'natureza_juridica', 'atividade_principal', 'atividade_secundaria_1', 'atividade_secundaria_2', 'atividade_secundaria_3', 'atividade_secundaria_4', 'atividade_secundaria_5', 'atividade_secundaria_6', 'atividade_secundaria_7', 'atividade_secundaria_8', 'atividade_secundaria_9', 'atividade_secundaria_10', 'atividade_secundaria_11', 'atividade_secundaria_12', 'atividade_secundaria_13', 'atividade_secundaria_14', 'atividade_secundaria_15', 'atividade_secundaria_16', 'atividade_secundaria_17', 'atividade_secundaria_18', 'atividade_secundaria_19', 'atividade_secundaria_20', 'atividade_secundaria_21', 'atividade_secundaria_22', 'atividade_secundaria_23', 'atividade_secundaria_24', 'atividade_secundaria_25', 'atividade_secundaria_26', 'atividade_secundaria_27', 'atividade_secundaria_28', 'atividade_secundaria_29', 'atividade_secundaria_30', 'atividade_secundaria_31', 'atividade_secundaria_32', 'atividade_secundaria_33', 'atividade_secundaria_34', 'atividade_secundaria_35', 'atividade_secundaria_36', 'atividade_secundaria_37', 'atividade_secundaria_38', 'atividade_secundaria_39', 'atividade_secundaria_40', 'atividade_secundaria_41', 'atividade_secundaria_42', 'atividade_secundaria_43', 'atividade_secundaria_44', 'atividade_secundaria_45', 'atividade_secundaria_46', 'atividade_secundaria_47', 'atividade_secundaria_48', 'atividade_secundaria_49', 'atividade_secundaria_50', 'atividade_secundaria_51', 'atividade_secundaria_52', 'atividade_secundaria_53', 'atividade_secundaria_54', 'atividade_secundaria_55', 'atividade_secundaria_56', 'atividade_secundaria_57', 'atividade_secundaria_58', 'atividade_secundaria_59', 'atividade_secundaria_60', 'atividade_secundaria_61', 'atividade_secundaria_62', 'atividade_secundaria_63', 'atividade_secundaria_64', 'atividade_secundaria_65', 'atividade_secundaria_66', 'atividade_secundaria_67', 'atividade_secundaria_68', 'atividade_secundaria_69', 'atividade_secundaria_70', 'atividade_secundaria_71', 'atividade_secundaria_72', 'atividade_secundaria_73', 'atividade_secundaria_74', 'atividade_secundaria_75', 'atividade_secundaria_76', 'atividade_secundaria_77', 'atividade_secundaria_78', 'atividade_secundaria_79', 'atividade_secundaria_80', 'atividade_secundaria_81', 'atividade_secundaria_82', 'atividade_secundaria_83', 'atividade_secundaria_84', 'atividade_secundaria_85', 'atividade_secundaria_86', 'atividade_secundaria_87', 'atividade_secundaria_88', 'atividade_secundaria_89', 'atividade_secundaria_90', 'atividade_secundaria_91', 'atividade_secundaria_92', 'atividade_secundaria_93', 'atividade_secundaria_94', 'atividade_secundaria_95', 'atividade_secundaria_96', 'atividade_secundaria_97', 'atividade_secundaria_98', 'atividade_secundaria_99'])

# Salva as informações em um arquivo CSV
empresas_df.to_csv(output_path, sep=';', index=False)